# Sort version  - Final version at the bottom

In [ ]:
# # FINAL - It works, map and DF with ctites sorted by distance towards the users city input

# In[7]:


# import the necessary libraries
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import folium
from IPython.display import HTML
import pandas as pd

# initialize Nominatim geocoder
geolocator = Nominatim(user_agent="my_app")

# prompt the user to enter a city and postal code
city = input("Please enter a city name: ") 
users_city_input = city
postal_code = input("Please enter the postal code: ")

# build the location string for the user location
location_str = f"{city}, {postal_code}"

# use geolocator to get the latitude and longitude of the user location
location = geolocator.geocode(location_str)

# create the map centered on the user location
if location is not None:
    user_lat = location.latitude
    user_lng = location.longitude
    myMap = folium.Map(location=[user_lat, user_lng], zoom_start=9)
    # add a marker for the user location
    folium.Marker([user_lat, user_lng], 
                  popup=f"Your Location: {location}",
                  icon=folium.Icon(color='green')).add_to(myMap)
else:
    print("\033[1;41;41mLocation not found or not available. Attention: This map doesn't show the location\033[0m")
    myMap = folium.Map(location=[48.137154, 11.576124], zoom_start=9)  # fallback to Munich's coordinates

# build the location strings and geocode them for each city
cities = ["Munich, Germany", "Frankfurt, Germany", "Hannover, Germany", "Bremen, Germany", "Hamburg, Germany","Offenbach, Germany"]
city_lats = []
city_lngs = []
for city in cities:
    location = geolocator.geocode(city)
    if location is not None:
        city_lats.append(location.latitude)
        city_lngs.append(location.longitude)
        # add a marker for the city location
        folium.Marker([location.latitude, location.longitude], 
                      popup=f"{city}",
                      icon=folium.Icon(color='blue')).add_to(myMap)
    else:
        print(f"\033[1;41;41m{city} not found or not available.\033[0m")

# calculate the distance between the user location and each city
if location is not None:
    for i in range(len(cities)):
        distance_km = round(geodesic((user_lat, user_lng), (city_lats[i], city_lngs[i])).km, 2)
        # add a line between the user location and the city
        folium.PolyLine(locations=[(user_lat, user_lng), (city_lats[i], city_lngs[i])], color='red').add_to(myMap)
        # display the distance on the map
        folium.Marker([(user_lat+city_lats[i])/2, (user_lng+city_lngs[i])/2], 
                      popup=f"Distance to {cities[i]}: {distance_km} km",
                      icon=folium.Icon(color='purple')).add_to(myMap)

        ######## Creating datframe          
        
# create an empty dataframe
df = pd.DataFrame()

# create a dataframe with data
for i in range(len(city_lats)):
    # create a new column with the name of the city
    city_name = cities[i]
    df[city_name] = [city_lats[i], city_lngs[i]]
    
    
# set the city names as the index
df.index = ['Latitude', 'Longitude']

# move the head of df to index
df = df.T.set_index(df.columns)

# rename columns with Lat and Lng values
df = df.rename(columns={'Latitude': 'Latitude', 'Longitude': 'Longitude'})


# calculate the distance between the user location and each city and store in a list
distance_km = []
if location is not None:
    for i in range(len(cities)):
        dist = round(geodesic((user_lat, user_lng), (city_lats[i], city_lngs[i])).km, 2)
        distance_km.append(dist)

        
        
df = df.assign(Distance_km=distance_km) 


# sort the DataFrame by shortest distance
df = df.sort_values(by=['Distance_km'])





#-------
import pandas as pd

# Crear un diccionario con los datos
data = {
    'bis KM': [100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700],
    'Stellplatz': [0,0,0,0,0,0,0,0,0,0,0],
    '2.5 ldm': [322.92, 352.56, 386.88, 444.60, 478.92, 538.20, 592.80, 648.96, 709.80, 851.76, 981.24],
    '4 ldm': [366.60, 399.36, 438.36, 503.88, 538.20, 614.64, 670.80, 736.32, 809.64, 959.40, 1104.48],
    '7 ldm': [452.40, 494.52, 541.32, 620.88, 663.00, 759.72, 828.36, 906.36, 996.84, 1184.04, 1368.12],
    'FTL': [583.44, 636.48, 697.32, 800.28, 859.56, 975.00, 1070.16, 1170.00, 1280.76, 1525.68, 1767.48]
}

# Crear un DataFrame a partir del diccionario
df_Prices_Type_A = pd.DataFrame(data)




# Transport Typ A+B (Transportstabil) kein gefährlicher Abfall

bis_km_values = df_Prices_Type_A["bis KM"]
distance = df["Distance_km"].iloc[0]
values_1 = []
values_2 = []

# create 2 diferent list than can be compared later and attahced to its corresponded mathc
# list for the min value
for value in bis_km_values:
    if value <= distance:
        values_1.append(value)
    else:
        values_1=[0]

# list for the max value
for value in bis_km_values:
    if distance <= value:
        values_2.append(value)
    else:
        values_2=[0]
# select lowest value from each least, in this case it would be the range between min-max belonging to the DF of distance KM from the user input     
valor_min= max(values_1) # max from the min list
valor_max = min(values_2) # min from the max list  


#--------
row_min = df_Prices_Type_A.loc[df_Prices_Type_A['bis KM'] >= valor_min]
row_max = df_Prices_Type_A.loc[df_Prices_Type_A['bis KM'] <= valor_max]

#--

row_min = row_min.drop(row_min.index[1:])  # you get a DF up to the shortest value (range)
row_max = df_Prices_Type_A.loc[df_Prices_Type_A['bis KM'] <= valor_max] # you get a DF up to the highest value (range)
row_max = row_max.iloc[-1:] 

#---
row_union_a = pd.concat([row_min, row_max]) # merge DF min and max values
row_union_a = row_union_a.reset_index() # the column index, gives the last postion from each row its in original DF
row_union_a['index'] = row_union_a['index'] + 1 # the number 0 is counts as 1 for the computer but not humans, therefore i add +1

#------------------------------------------------------------------------------------------------------------------------------

import pandas as pd

data = {'bis KM': [100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700],
        'Stellplatz': [0,0,0,0,0,0,0,0,0,0,0],
        "2.5 ldm": [2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, None],
        '4 ldm': [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, None],
        '7 ldm': [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, None],
        'FTL': [530.40, 653.64, 814.32, 978.12, 1162.20, 1343.16, 1528.80, 1703.52, 1884.48, 2238.60, 3511.56]}

df_Prices_Type_C = pd.DataFrame(data)



# Transport Typ C (Defekt und nicht transportstabil) gefährlicher Abfall + SV 376

bis_km_values = df_Prices_Type_C["bis KM"]
distance = df["Distance_km"].iloc[0]
values_1C = []
values_2C = []

# create 2 diferent list than can be compared later and attahced to its corresponded mathc
# list for the min value
for value in bis_km_values:
    if value <= distance:
        values_1C.append(value)
    else:
        values_1C=[0]

# list for the max value
for value in bis_km_values:
    if distance <= value:
        values_2C.append(value)
    else:
        values_2C=[0]
# select lowest value from each least, in this case it would be the range between min-max belonging to the DF of distance KM from the user input     
valor_minC= max(values_1C) # max from the min list
valor_maxC = min(values_2C) # min from the max list  


#--------
row_minC = df_Prices_Type_C.loc[df_Prices_Type_C['bis KM'] >= valor_minC]
row_maxC = df_Prices_Type_C.loc[df_Prices_Type_C['bis KM'] <= valor_maxC]

#--

row_minC = row_minC.drop(row_minC.index[1:])  # you get a DF up to the shortest value (range)
row_maxC = df_Prices_Type_C.loc[df_Prices_Type_C['bis KM'] <= valor_maxC] # you get a DF up to the highest value (range)
row_maxC = row_maxC.iloc[-1:] 

#---
row_unionC = pd.concat([row_minC, row_maxC]) # merge DF min and max values
row_unionC= row_unionC.reset_index() # the column index, gives the last postion from each row its in original DF
row_unionC['index'] = row_unionC['index'] + 1 # the number 0 is counts as 1 for the computer but not humans, therefore i add +1

#----
print("Table with distances to " +" " + users_city_input)
print("")
### Map and Df can be only together posted if DF is in print mode
print(df)
print("") #space
print("") #space

print("Transport Typ A+B (Transportstabil) kein gefährlicher Abfall")
print("") #space
print(row_union_a) 
print("--------------------------------------------------------------")
print("")
print("Transport Typ C (Defekt und nicht transportstabil) gefährlicher Abfall + SV 376")
print("")
print(row_unionC)



# display the map
HTML(myMap._repr_html_())

In [ ]:
Please enter a city name: Wiesbaden
Please enter the postal code: 65189


## Code with the both complete lists

In [ ]:
# # FINAL - It works, map and DF with ctites sorted by distance towards the users city input

# In[7]:


# import the necessary libraries
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import folium
from IPython.display import HTML
import pandas as pd

# initialize Nominatim geocoder
geolocator = Nominatim(user_agent="my_app")

# prompt the user to enter a city and postal code
city = input("Please enter a city name: ") 
users_city_input = city
postal_code = input("Please enter the postal code: ")

# build the location string for the user location
location_str = f"{city}, {postal_code}"

# use geolocator to get the latitude and longitude of the user location
location = geolocator.geocode(location_str)

# create the map centered on the user location
if location is not None:
    user_lat = location.latitude
    user_lng = location.longitude
    myMap = folium.Map(location=[user_lat, user_lng], zoom_start=9)
    # add a marker for the user location
    folium.Marker([user_lat, user_lng], 
                  popup=f"Your Location: {location}",
                  icon=folium.Icon(color='green')).add_to(myMap)
else:
    print("\033[1;41;41mLocation not found or not available. Attention: This map doesn't show the location\033[0m")
    myMap = folium.Map(location=[48.137154, 11.576124], zoom_start=9)  # fallback to Munich's coordinates

# build the location strings and geocode them for each city
cities = ["Munich, Germany", "Frankfurt, Germany", "Hannover, Germany", "Bremen, Germany", "Hamburg, Germany","Offenbach, Germany"]
city_lats = []
city_lngs = []
for city in cities:
    location = geolocator.geocode(city)
    if location is not None:
        city_lats.append(location.latitude)
        city_lngs.append(location.longitude)
        # add a marker for the city location
        folium.Marker([location.latitude, location.longitude], 
                      popup=f"{city}",
                      icon=folium.Icon(color='blue')).add_to(myMap)
    else:
        print(f"\033[1;41;41m{city} not found or not available.\033[0m")

# calculate the distance between the user location and each city
if location is not None:
    for i in range(len(cities)):
        distance_km = round(geodesic((user_lat, user_lng), (city_lats[i], city_lngs[i])).km, 2)
        # add a line between the user location and the city
        folium.PolyLine(locations=[(user_lat, user_lng), (city_lats[i], city_lngs[i])], color='red').add_to(myMap)
        # display the distance on the map
        folium.Marker([(user_lat+city_lats[i])/2, (user_lng+city_lngs[i])/2], 
                      popup=f"Distance to {cities[i]}: {distance_km} km",
                      icon=folium.Icon(color='purple')).add_to(myMap)

        ######## Creating datframe          
        
# create an empty dataframe
df = pd.DataFrame()

# create a dataframe with data
for i in range(len(city_lats)):
    # create a new column with the name of the city
    city_name = cities[i]
    df[city_name] = [city_lats[i], city_lngs[i]]
    
    
# set the city names as the index
df.index = ['Latitude', 'Longitude']

# move the head of df to index
df = df.T.set_index(df.columns)

# rename columns with Lat and Lng values
df = df.rename(columns={'Latitude': 'Latitude', 'Longitude': 'Longitude'})


# calculate the distance between the user location and each city and store in a list
distance_km = []
if location is not None:
    for i in range(len(cities)):
        dist = round(geodesic((user_lat, user_lng), (city_lats[i], city_lngs[i])).km, 2)
        distance_km.append(dist)

        
        
df = df.assign(Distance_km=distance_km) 


# sort the DataFrame by shortest distance
df = df.sort_values(by=['Distance_km'])





#-------
import pandas as pd

# Crear un diccionario con los datos
data = {
    'bis KM': [100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700],
    'Stellplatz': [0,0,0,0,0,0,0,0,0,0,0],
    '2.5 ldm': [322.92, 352.56, 386.88, 444.60, 478.92, 538.20, 592.80, 648.96, 709.80, 851.76, 981.24],
    '4 ldm': [366.60, 399.36, 438.36, 503.88, 538.20, 614.64, 670.80, 736.32, 809.64, 959.40, 1104.48],
    '7 ldm': [452.40, 494.52, 541.32, 620.88, 663.00, 759.72, 828.36, 906.36, 996.84, 1184.04, 1368.12],
    'FTL': [583.44, 636.48, 697.32, 800.28, 859.56, 975.00, 1070.16, 1170.00, 1280.76, 1525.68, 1767.48]
}

# Crear un DataFrame a partir del diccionario
df_Prices_Type_A = pd.DataFrame(data)





#------------------------------------------------------------------------------------------------------------------------------

import pandas as pd

data = {'bis KM': [100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700],
        'Stellplatz': [0,0,0,0,0,0,0,0,0,0,0],
        "2.5 ldm": [2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, None],
        '4 ldm': [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, None],
        '7 ldm': [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, None],
        'FTL': [530.40, 653.64, 814.32, 978.12, 1162.20, 1343.16, 1528.80, 1703.52, 1884.48, 2238.60, 3511.56]}

df_Prices_Type_C = pd.DataFrame(data)




#----
print("Table with distances to " +" " + users_city_input)
print("")
### Map and Df can be only together posted if DF is in print mode
print(df)
print("") #space
print("") #space

print("Transport Typ A+B (Transportstabil) kein gefährlicher Abfall")
print("") #space
print(df_Prices_Type_A ) 
print("--------------------------------------------------------------")
print("")
print("Transport Typ C (Defekt und nicht transportstabil) gefährlicher Abfall + SV 376")
print("")
print(df_Prices_Type_C)



# display the map
HTML(myMap._repr_html_())

### Import files

In [ ]:
df_excel_test_A = pd.read_excel('Transport Typ A+B.xlsx', sheet_name='Sheet1')

In [ ]:
df_excel_test_A 

In [ ]:
df_excel_test_B = pd.read_csv('Transport Typ A+B.csv')

In [ ]:
df_excel_test_B

In [ ]:
# remove the first row and reset the header
new_header = df_excel_test_A.iloc[0] # save the current header as a new variable
df_excel_test_A= df_excel_test_A[1:] # remove the first row
df_excel_test_A.columns = new_header # set the new header as the old second row

In [ ]:
df_excel_test_A

## code with DF imported - (only the first price list is imported) 
### Example used with df_excel_test_B 

In [ ]:
# # FINAL - It works, map and DF with ctites sorted by distance towards the users city input

# In[7]:


# import the necessary libraries
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import folium
from IPython.display import HTML
import pandas as pd

# initialize Nominatim geocoder
geolocator = Nominatim(user_agent="my_app")

# prompt the user to enter a city and postal code
city = input("Please enter a city name: ") 
users_city_input = city
postal_code = input("Please enter the postal code: ")

# build the location string for the user location
location_str = f"{city}, {postal_code}"

# use geolocator to get the latitude and longitude of the user location
location = geolocator.geocode(location_str)

# create the map centered on the user location
if location is not None:
    user_lat = location.latitude
    user_lng = location.longitude
    myMap = folium.Map(location=[user_lat, user_lng], zoom_start=9)
    # add a marker for the user location
    folium.Marker([user_lat, user_lng], 
                  popup=f"Your Location: {location}",
                  icon=folium.Icon(color='green')).add_to(myMap)
else:
    print("\033[1;41;41mLocation not found or not available. Attention: This map doesn't show the location\033[0m")
    myMap = folium.Map(location=[48.137154, 11.576124], zoom_start=9)  # fallback to Munich's coordinates

# build the location strings and geocode them for each city
cities = ["Munich, Germany", "Frankfurt, Germany", "Hannover, Germany", "Bremen, Germany", "Hamburg, Germany","Offenbach, Germany"]
city_lats = []
city_lngs = []
for city in cities:
    location = geolocator.geocode(city)
    if location is not None:
        city_lats.append(location.latitude)
        city_lngs.append(location.longitude)
        # add a marker for the city location
        folium.Marker([location.latitude, location.longitude], 
                      popup=f"{city}",
                      icon=folium.Icon(color='blue')).add_to(myMap)
    else:
        print(f"\033[1;41;41m{city} not found or not available.\033[0m")

# calculate the distance between the user location and each city
if location is not None:
    for i in range(len(cities)):
        distance_km = round(geodesic((user_lat, user_lng), (city_lats[i], city_lngs[i])).km, 2)
        # add a line between the user location and the city
        folium.PolyLine(locations=[(user_lat, user_lng), (city_lats[i], city_lngs[i])], color='red').add_to(myMap)
        # display the distance on the map
        folium.Marker([(user_lat+city_lats[i])/2, (user_lng+city_lngs[i])/2], 
                      popup=f"Distance to {cities[i]}: {distance_km} km",
                      icon=folium.Icon(color='purple')).add_to(myMap)

        ######## Creating datframe          
        
# create an empty dataframe
df = pd.DataFrame()

# create a dataframe with data
for i in range(len(city_lats)):
    # create a new column with the name of the city
    city_name = cities[i]
    df[city_name] = [city_lats[i], city_lngs[i]]
    
    
# set the city names as the index
df.index = ['Latitude', 'Longitude']

# move the head of df to index
df = df.T.set_index(df.columns)

# rename columns with Lat and Lng values
df = df.rename(columns={'Latitude': 'Latitude', 'Longitude': 'Longitude'})


# calculate the distance between the user location and each city and store in a list
distance_km = []
if location is not None:
    for i in range(len(cities)):
        dist = round(geodesic((user_lat, user_lng), (city_lats[i], city_lngs[i])).km, 2)
        distance_km.append(dist)

        
        
df = df.assign(Distance_km=distance_km) 


# sort the DataFrame by shortest distance
df = df.sort_values(by=['Distance_km'])





#-------
import pandas as pd


# Crear un DataFrame a partir del diccionario
df_Prices_Type_A = df_excel_test_B





#------------------------------------------------------------------------------------------------------------------------------

import pandas as pd

data = {'bis KM': [100, 150, 200, 250, 300, 350, 400, 450, 500, 600, 700],
        'Stellplatz': [0,0,0,0,0,0,0,0,0,0,0],
        "2.5 ldm": [2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, 2.5, None],
        '4 ldm': [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, None],
        '7 ldm': [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, None],
        'FTL': [530.40, 653.64, 814.32, 978.12, 1162.20, 1343.16, 1528.80, 1703.52, 1884.48, 2238.60, 3511.56]}

df_Prices_Type_C = pd.DataFrame(data)




#----
print("Table with distances to " +" " + users_city_input)
print("")
### Map and Df can be only together posted if DF is in print mode
print(df)
print("") #space
print("") #space

print("Transport Typ A+B (Transportstabil) kein gefährlicher Abfall")
print("") #space
print(df_Prices_Type_A ) 
print("--------------------------------------------------------------")
print("")
print("Transport Typ C (Defekt und nicht transportstabil) gefährlicher Abfall + SV 376")
print("")
print(df_Prices_Type_C)



# display the map
HTML(myMap._repr_html_())

## Choose what DFs you want.
#### Opcion A: Transport_nach_WP
#### Opcion B: Transport_nach_LTS

In [ ]:
print("What list do you want? ")
print("Option A: Transport_nach_WP")
print("Option B: Transport_nach_LTS")
option = input()

if option == "a" or option == "A":
 print("You choose option A")
  # Aquí iría el resultado si se escogió la opción A

elif option == "B" or option ==  "b":
  print("You choose option B")
  # Aquí iría el resultado si se escogió la opción B

    
else:
  print("\033[1;41;41mInvalid option. Please enter a or b.\033[0m")



In [ ]:
print("What list do you want? ")
print("Option A: Transport_nach_LTS")
print("Option B: Transport_nach_WP ")
option = input()


if option == "a" or option == "A":
    print("You choose option A")
    
    df_Prices_Type_A = pd.read_excel('Transport Typ A+B.xlsx', sheet_name='Sheet1')
    
    new_header = df_Prices_Type_A.iloc[0] # save the current header as a new variable
    df_Prices_Type_A= df_Prices_Type_A[1:] # remove the first row
    df_Prices_Type_A.columns = new_header # set the new header as the old second row
   

    df_Prices_Type_C = pd.read_excel('Transport Typ C (Defekt und nicht transportstabil) gefährlicher Abfall + SV 376.xlsx', sheet_name='Sheet1')
  
    new_header = df_Prices_Type_C.iloc[0] # save the current header as a new variable
    df_Prices_Type_C= df_Prices_Type_C[1:] # remove the first row
    df_Prices_Type_C.columns = new_header # set the new header as the old second row
    


elif option == "B" or option ==  "b":
    print("You choose option B")
        
    df_Prices_Type_A = pd.read_excel('WP_Transport Typ A+B (Transportstabil) .xlsx', sheet_name='Sheet3')
    
    new_header = df_Prices_Type_A.iloc[0] # save the current header as a new variable

    df_Prices_Type_A= df_Prices_Type_A[1:] # remove the first row
    df_Prices_Type_A.columns = new_header # set the new header as the old second row
   

    df_Prices_Type_C = pd.read_excel('WP_Transport Typ C (Defekt und nicht transportstabil) .xlsx', sheet_name='Sheet3')
  
    new_header = df_Prices_Type_C.iloc[0] # save the current header as a new variable
    df_Prices_Type_C= df_Prices_Type_C[1:] # remove the first row
    df_Prices_Type_C.columns = new_header # set the new header as the old second row

    
else:
    print("\033[1;41;41mInvalid option. Please enter a or b.\033[0m")

# Final
##### Users can independently upload Excel files to Jupyter Notebook and then simply run the necessary code to process the data.

In [7]:
import pandas as pd

In [8]:
!pip install --upgrade pandas

In [2]:
pip install pandas

  Using cached pandas-2.0.1-cp310-cp310-win_amd64.whl (10.7 MB)
  Using cached numpy-1.24.3-cp310-cp310-win_amd64.whl (14.8 MB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install openpyxl

  Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install geopy


  Using cached geopy-2.3.0-py3-none-any.whl (119 kB)
  Using cached geographiclib-2.0-py3-none-any.whl (40 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install folium


  Using cached folium-0.14.0-py2.py3-none-any.whl (102 kB)
  Using cached branca-0.6.0-py3-none-any.whl (24 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install geopy


In [4]:
import pandas as pd 
import sys
print("What list do you want? ")
print("Option A: Transport_nach_LTS")
print("Option B: Transport_nach_WP ")
option = input()


if option == "a" or option == "A":
    print("You choose option A")
    
    df_Prices_Type_A = pd.read_excel('Transport Typ A+B.xlsx', sheet_name='Sheet1')
    
    new_header = df_Prices_Type_A.iloc[0] # save the current header as a new variable
    df_Prices_Type_A= df_Prices_Type_A[1:] # remove the first row
    df_Prices_Type_A.columns = new_header # set the new header as the old second row
   

    df_Prices_Type_C = pd.read_excel('Transport Typ C (Defekt und nicht transportstabil) gefährlicher Abfall + SV 376.xlsx', sheet_name='Sheet1')
  
    new_header = df_Prices_Type_C.iloc[0] # save the current header as a new variable
    df_Prices_Type_C= df_Prices_Type_C[1:] # remove the first row
    df_Prices_Type_C.columns = new_header # set the new header as the old second row
    


elif option == "B" or option ==  "b":
    print("You choose option B")
        
    df_Prices_Type_A = pd.read_excel('WP_Transport Typ A+B (Transportstabil) .xlsx', sheet_name='Sheet3')
    
    new_header = df_Prices_Type_A.iloc[0] # save the current header as a new variable

    df_Prices_Type_A= df_Prices_Type_A[1:] # remove the first row
    df_Prices_Type_A.columns = new_header # set the new header as the old second row
   

    df_Prices_Type_C = pd.read_excel('WP_Transport Typ C (Defekt und nicht transportstabil) .xlsx', sheet_name='Sheet3')
  
    new_header = df_Prices_Type_C.iloc[0] # save the current header as a new variable
    df_Prices_Type_C= df_Prices_Type_C[1:] # remove the first row
    df_Prices_Type_C.columns = new_header # set the new header as the old second row

    
else:
    print("\033[1;41;41mInvalid option. Please enter A/a or B/b.\033[0m")
    sys.exit()
    
#------------------------------------------------------------------



# # FINAL - It works, map and DF with ctites sorted by distance towards the users city input

# In[7]:


# import the necessary libraries
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import folium
from IPython.display import HTML
import pandas as pd

# initialize Nominatim geocoder
geolocator = Nominatim(user_agent="my_app")

# prompt the user to enter a city and postal code
city = input("Please enter a city name: ") 
users_city_input = city
postal_code = input("Please enter the postal code: ")

# build the location string for the user location
location_str = f"{city}, {postal_code}"

# use geolocator to get the latitude and longitude of the user location
location = geolocator.geocode(location_str)

# create the map centered on the user location
if location is not None:
    user_lat = location.latitude
    user_lng = location.longitude
    myMap = folium.Map(location=[user_lat, user_lng], zoom_start=9)
    # add a marker for the user location
    folium.Marker([user_lat, user_lng], 
                  popup=f"Your Location: {location}",
                  icon=folium.Icon(color='green')).add_to(myMap)
else:
    print("\033[1;41;41mLocation not found or not available. Attention: This map doesn't show the location\033[0m")
    myMap = folium.Map(location=[48.137154, 11.576124], zoom_start=9)  # fallback to Munich's coordinates

# build the location strings and geocode them for each city
cities = ["Tirana, Albania", "Andorra la Vella, Andorra", "Yerevan, Armenia", "Vienna, Austria", "Minsk, Belarus", "Brussels, Belgium", 
          "Sarajevo, Bosnia and Herzegovina", "Sofia, Bulgaria", "Zagreb, Croatia", "Nicosia, Cyprus", "Prague, Czech Republic", 
          "Copenhagen, Denmark", "Tallinn, Estonia", "Helsinki, Finland", "Paris, France", "Berlin, Germany", "Gibraltar, Gibraltar", 
          "Athens, Greece", "Budapest, Hungary", "Reykjavik, Iceland", "Dublin, Ireland", "Rome, Italy", "Nur-Sultan, Kazakhstan", 
          "Pristina, Kosovo", "Riga, Latvia", "Vilnius, Lithuania", "Luxembourg, Luxembourg", "Skopje, Macedonia", "Valletta, Malta", 
          "Chisinau, Moldova", "Podgorica, Montenegro", "Amsterdam, Netherlands", "Oslo, Norway", "Warsaw, Poland", "Lisbon, Portugal", 
          "Bucharest, Romania", "Moscow, Russia", "Belgrade, Serbia", "Bratislava, Slovakia", "Ljubljana, Slovenia", "Madrid, Spain", 
          "Stockholm, Sweden", "Bern, Switzerland", "London, UK", "Kyiv, Ukraine", "Cayenne, French Guiana", "Basse-Terre, Guadeloupe", 
          "Fort-de-France, Martinique", "Saint-Denis, Reunion"]
city_lats = []
city_lngs = []
for city in cities:
    location = geolocator.geocode(city)
    if location is not None:
        city_lats.append(location.latitude)
        city_lngs.append(location.longitude)
        # add a marker for the city location
        folium.Marker([location.latitude, location.longitude], 
                      popup=f"{city}",
                      icon=folium.Icon(color='blue')).add_to(myMap)
    else:
        print(f"\033[1;41;41m{city} not found or not available.\033[0m")

# calculate the distance between the user location and each city
if location is not None:
    for i in range(len(cities)):
        distance_km = round(geodesic((user_lat, user_lng), (city_lats[i], city_lngs[i])).km, 2)
        # add a line between the user location and the city
        folium.PolyLine(locations=[(user_lat, user_lng), (city_lats[i], city_lngs[i])], color='red').add_to(myMap)
        # display the distance on the map
        folium.Marker([(user_lat+city_lats[i])/2, (user_lng+city_lngs[i])/2], 
                      popup=f"Distance to {cities[i]}: {distance_km} km",
                      icon=folium.Icon(color='purple')).add_to(myMap)

        ######## Creating datframe          
        
# create an empty dataframe
df = pd.DataFrame()

# create a dataframe with data
for i in range(len(city_lats)):
    # create a new column with the name of the city
    city_name = cities[i]
    df[city_name] = [city_lats[i], city_lngs[i]]
    
    
# set the city names as the index
df.index = ['Latitude', 'Longitude']

# move the head of df to index
df = df.T.set_index(df.columns)

# rename columns with Lat and Lng values
df = df.rename(columns={'Latitude': 'Latitude', 'Longitude': 'Longitude'})


# calculate the distance between the user location and each city and store in a list
distance_km = []
if location is not None:
    for i in range(len(cities)):
        dist = round(geodesic((user_lat, user_lng), (city_lats[i], city_lngs[i])).km, 2)
        distance_km.append(dist)

        
        
df = df.assign(Distance_km=distance_km) 


# sort the DataFrame by shortest distance
df = df.sort_values(by=['Distance_km'])



#------------------------------------------------------------------------------------------------------------------------------


#----
print("Table with distances to " +" " + users_city_input)
print("")
### Map and Df can be only together posted if DF is in print mode
print(df)
print("") #space
print("") #space

print("Transport Typ A+B (Transportstabil) kein gefährlicher Abfall")
print("") #space
print(df_Prices_Type_A ) 
print("--------------------------------------------------------------")
print("")
print("Transport Typ C (Defekt und nicht transportstabil) gefährlicher Abfall + SV 376")
print("")
print(df_Prices_Type_C)


# display the map
HTML(myMap._repr_html_())

What list do you want? 
Option A: Transport_nach_LTS
Option B: Transport_nach_WP 
a
You choose option A
Please enter a city name: berlin
Please enter the postal code: 10115
Table with distances to  berlin

                                   Latitude  Longitude  Distance_km
Berlin, Germany                   52.517037  13.388860         0.00
Prague, Czech Republic            50.087465  14.421254       279.72
Copenhagen, Denmark               55.686724  12.570072       356.84
Warsaw, Poland                    52.231958  21.006725       519.53
Vienna, Austria                   48.208354  16.372504       524.08
Bratislava, Slovakia              48.151699  17.109306       552.95
Amsterdam, Netherlands            52.373080   4.892453       577.60
Luxembourg, Luxembourg            49.611277   6.129799       602.43
Brussels, Belgium                 50.846557   4.351697       651.59
Budapest, Hungary                 47.481390  19.146094       695.30
Ljubljana, Slovenia               46.050027  1

In [10]:
 df_Prices_Type_A = pd.read_excel('Transport Typ A+B.xlsx', sheet_name='Sheet1')

AttributeError: module 'pandas' has no attribute 'read_excel'

In [ ]:
# different print options

In [ ]:
print("\033[93;101m\033[1mInvalid option. Please enter a or b.\033[0m")



In [ ]:
print("\033[1;41;41mLocation not found\033[0m")

In [ ]:
print("\033[1;38;40mLocation not found\033[0m")

In [ ]:
print("\033[1;30;40mLocation not found\033[0m")

In [ ]:
print("\033[1;41;41mLocation not found or not available. Attention: This map doesn't show the location\033[0m")

### Give the option of asking how many cities you wanna check, afterwards the cties and PCs will be asked and added to a map

In [6]:
x = int(input())
print("Your input")
print("")
for i in range(x):
    print(i+1)


3
Your input

1
2
3


In [7]:
x = int(input())
print("Your input")
print("")
for i in range(x):
    print(i+1)

    option = input("Which option do you prefer, A or B? ")
    if option.lower() == "a":
        question = "What is your favorite color?"
    elif option.lower() == "b":
        question = "What is your favorite food?"
    else:
        print("Invalid input")
        continue

    answer = input(question)
    print("Your answer is:", answer)
    print("")




5
Your input

1
Which option do you prefer, A or B? b
What is your favorite food?2
Your answer is: 2

2
Which option do you prefer, A or B? a
What is your favorite color?a
Your answer is: a

3
Which option do you prefer, A or B? a
What is your favorite color?a
Your answer is: a

4
Which option do you prefer, A or B? a
What is your favorite color?a
Your answer is: a

5
Which option do you prefer, A or B? a
What is your favorite color?a
Your answer is: a



In [8]:
import pandas as pd

df = pd.DataFrame()

while True:
    # Ask the user to input a SKU and a number
    x = input("Enter SKU: ")
    n_sku = int(input("Enter a number: "))

    try:
        # Add the SKU and number to the DataFrame
        new_row = {'Config_SKU': x, "Quantity":n_sku}
        df = pd.concat([df, pd.DataFrame(new_row, index=[0])], ignore_index=True)
        
        # Ask the user if they want to add another SKU
        add_more = input("Do you want to add another SKU? yes/no ")
        
        # If the user doesn't want to add more SKUs, break out of the loop
        if add_more == "no" or add_more == "n" or add_more == "NO" or add_more == "N":
            break
    
    except ValueError:
        # If the input could not be converted to an integer, print an error message
        print("Invalid input. Please enter a number.")    

df      


Enter SKU: a
Enter a number: 1
Do you want to add another SKU? yes/no y
Enter SKU: b
Enter a number: 2
Do you want to add another SKU? yes/no y
Enter SKU: c
Enter a number: 3
Do you want to add another SKU? yes/no n


,Config_SKU,Quantity
0,a,1
1,b,2
2,c,3


In [2]:
# build the location strings and geocode them for each city
cities = ["Tirana, Albania", "Andorra la Vella, Andorra", "Yerevan, Armenia", "Vienna, Austria", "Minsk, Belarus", "Brussels, Belgium", 
          "Sarajevo, Bosnia and Herzegovina", "Sofia, Bulgaria", "Zagreb, Croatia", "Nicosia, Cyprus", "Prague, Czech Republic", 
          "Copenhagen, Denmark", "Tallinn, Estonia", "Helsinki, Finland", "Paris, France", "Berlin, Germany", "Gibraltar, Gibraltar", 
          "Athens, Greece", "Budapest, Hungary", "Reykjavik, Iceland", "Dublin, Ireland", "Rome, Italy", "Nur-Sultan, Kazakhstan", 
          "Pristina, Kosovo", "Riga, Latvia", "Vilnius, Lithuania", "Luxembourg, Luxembourg", "Skopje, Macedonia", "Valletta, Malta", 
          "Chisinau, Moldova", "Podgorica, Montenegro", "Amsterdam, Netherlands", "Oslo, Norway", "Warsaw, Poland", "Lisbon, Portugal", 
          "Bucharest, Romania", "Moscow, Russia", "Belgrade, Serbia", "Bratislava, Slovakia", "Ljubljana, Slovenia", "Madrid, Spain", 
          "Stockholm, Sweden", "Bern, Switzerland", "London, UK", "Kyiv, Ukraine", "Cayenne, French Guiana", "Basse-Terre, Guadeloupe", 
          "Fort-de-France, Martinique", "Saint-Denis, Reunion"]

city_lats = []
city_lngs = []
for city in cities:
    location = geolocator.geocode(city)
    if location is not None:
        city_lats.append(location.latitude)
        city_lngs.append(location.longitude)
        # add a marker for the city location
        folium.Marker([location.latitude, location.longitude], 
                      popup=f"{city}: {location.latitude}, {location.longitude}",
                      icon=folium.Icon(color='blue')).add_to(myMap)
    else:
        print(f"\033[1;41;41mLocation not found for {city}\033[0m")


In [3]:
import pandas as pd 
import sys
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import folium

# initialize Nominatim geocoder
geolocator = Nominatim(user_agent="my_app")

# prompt the user to enter a city and postal code
city = input("Please enter a city name: ") 
users_city_input = city
postal_code = input("Please enter the postal code: ")

# build the location string for the user location
location_str = f"{city}, {postal_code}"

# use geolocator to get the latitude and longitude of the user location
location = geolocator.geocode(location_str)

# create the map centered on the user location
if location is not None:
    user_lat = location.latitude
    user_lng = location.longitude
    myMap = folium.Map(location=[user_lat, user_lng], zoom_start=9)
    # add a marker for the user location
    folium.Marker([user_lat, user_lng], 
                  popup=f"Your Location: {location}",
                  icon=folium.Icon(color='green')).add_to(myMap)
else:
    print("\033[1;41;41mLocation not found or not available. Attention: This map doesn't show the location\033[0m")
    myMap = folium.Map(location=[48.137154, 11.576124], zoom_start=9)  # fallback to Munich's coordinates

# build the location strings and geocode them for each city
cities = ["Tirana, Albania", "Andorra la Vella, Andorra", "Yerevan, Armenia", "Vienna, Austria", 
          "Minsk, Belarus", "Brussels, Belgium", "Sarajevo, Bosnia and Herzegovina", "Sofia, Bulgaria",
          "Zagreb, Croatia", "Nicosia, Cyprus", "Prague, Czech Republic", "Copenhagen, Denmark", 
          "Tallinn, Estonia", "Helsinki, Finland", "Paris, France", "Berlin, Germany", "Gibraltar", 
          "Athens, Greece", "Budapest, Hungary", "Reykjavik, Iceland", "Dublin, Ireland", "Rome, Italy", 
          "Nur-Sultan, Kazakhstan", "Pristina, Kosovo", "Riga, Latvia", "Vilnius, Lithuania", 
          "Luxembourg, Luxembourg", "Skopje, North Macedonia", "Valletta, Malta", "Chișinău, Moldova", 
          "Podgorica, Montenegro", "Amsterdam, Netherlands", "Oslo, Norway", "Warsaw, Poland", 
          "Lisbon, Portugal", "Bucharest, Romania", "Moscow, Russia", "Belgrade, Serbia", 
          "Bratislava, Slovakia", "Ljubljana, Slovenia", "Madrid, Spain", "Stockholm, Sweden", 
          "Bern, Switzerland", "London, UK", "Kiev, Ukraine", "Cayenne, French Guiana", 
          "Basse-Terre, Guadeloupe", "Fort-de-France, Martinique", "Saint-Denis, Reunion"]

city_lats = []
city_lngs = []
for city in cities:
    location = geolocator.geocode(city)
    if location is not None:
        city_lats.append(location.latitude)
        city_lngs.append(location.longitude)
        # add a marker for the


Please enter a city name: berlin
Please enter the postal code: 10115
